In [128]:
import pandas as pd

# Set display options to show more rows and columns
pd.set_option('display.max_columns', None) # Show all columns

listings = pd.read_csv('../AirBnb datasets/barcelona/listings.csv')

listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,269467,https://www.airbnb.com/rooms/269467,20240615011618,2024-06-15,city scrape,Sunny and quiet Loft next to Sagrada Familia,The apartment is a cosy open space with 65 m2....,NaN,https://a0.muscache.com/pictures/a6445a90-94eb...,1411775,https://www.airbnb.com/users/show/1411775,Jeremies,2011-11-16,"Barcelona, Spain","We are Ok Apartment Barcelona, a local agency ...",within an hour,100%,88%,f,https://a0.muscache.com/im/pictures/user/1f974...,https://a0.muscache.com/im/pictures/user/1f974...,La Sagrada Família,5.0,7.0,"['email', 'phone']",t,t,NaN,la Sagrada Família,Eixample,41.401810,2.17610,Entire loft,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Kitchen"", ""Patio or balcony"", ""Host greets y...",$176.00,7,760,7,7,760,760,7.0,760.0,NaN,t,0,0,0,257,2024-06-15,90,17,1,2018-01-01,2024-05-22,4.74,4.87,4.59,4.83,4.79,4.81,4.67,HUTB-001129,t,3,3,0,0,1.15
1,270197,https://www.airbnb.com/rooms/270197,20240615011618,2024-06-15,city scrape,*monthly stays* 4 Bedroom Apartment near Rambla,Rented only for monthly periods. 4 bedroom apa...,NaN,https://a0.muscache.com/pictures/69907440-d8df...,1414702,https://www.airbnb.com/users/show/1414702,Ferran,2011-11-16,"Barcelona, Spain","Hello, everyone!\n\nBorn and bred in Barcelona...",within an hour,98%,97%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,El Raval,36.0,112.0,"['email', 'phone']",t,t,NaN,el Raval,Ciutat Vella,41.379360,2.17376,Entire rental unit,Entire home/apt,6,1.5,1.5 baths,4.0,5.0,"[""Kitchen"", ""Washer"", ""Smoking allowed"", ""Wifi...",$74.00,32,760,32,32,760,760,32.0,760.0,NaN,t,0,0,0,0,2024-06-15,2,0,0,2016-07-03,2016-08-13,4.00,5.00,4.50,5.00,5.00,5.00,4.50,N/A SOLO ESTANCIAS MENSUALES,f,27,27,0,0,0.02
2,18674,https://www.airbnb.com/rooms/18674,20240615011618,2024-06-15,city scrape,Huge flat for 8 people close to Sagrada Familia,110m2 apartment to rent in Barcelona. Located ...,Apartment in Barcelona located in the heart of...,https://a0.muscache.com/pictures/13031453/413c...,71615,https://www.airbnb.com/users/show/71615,Mireia Maria,2010-01-19,"Barcelona, Spain","We are Mireia (47) & Maria (49), two multiling...",within an hour,95%,85%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,la Sagrada Família,41.0,45.0,"['email', 'phone']",t,t,"Barcelona, CT, Spain",la Sagrada Família,Eixample,41.405560,2.17262,Entire rental unit,Entire home/apt,8,2.0,2 baths,3.0,6.0,"[""Pack \u2019n play/Travel crib"", ""Free street...",$257.00,1,1125,1,4,1125,1125,2.1,1125.0,NaN,t,10,36,66,177,2024-06-15,41,5,0,2013-05-27,2024-05-02,4.35,4.45,4.63,4.75,4.68,4.78,4.30,HUTB-002062,t,27,27,

In [129]:
# Identify columns where all values are the same
constant_columns = [col for col in listings.columns if listings[col].nunique() == 1]

# Print the columns where all values are the same
print("Columns where all values are the same:", constant_columns)

Columns where all values are the same: ['scrape_id', 'last_scraped', 'calendar_last_scraped']


In [130]:
# Remove columns where all values are the same
columns_to_remove = ['scrape_id', 'last_scraped', 'calendar_last_scraped']
listings = listings.drop(columns=columns_to_remove)

In [131]:
# Remove irrelevant or uninformative features
columns_to_remove = ['listing_url', 'source', 'name', 'description', 'neighborhood_overview',
                     'picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url',
                     'host_picture_url', 'license', 'calendar_updated']

listings = listings.drop(columns=columns_to_remove)

In [132]:
# Convert boolean values into binary representations
listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
listings['host_has_profile_pic'] = listings['host_has_profile_pic'].replace({'t': 1, 'f': 0})
listings['host_identity_verified'] = listings['host_identity_verified'].replace({'t': 1, 'f': 0})
listings['has_availability'] = listings['has_availability'].replace({'t': 1, 'f': 0})

/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_77984/25808645.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_77984/25808645.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_77984/25808645.py:4: FutureWarning: Downcasting beh

In [133]:
# Remove the dollar symbol and commas, then convert the column to numeric
listings['price'] = listings['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float)

In [134]:
# Handle Date columns (convert to datetime, then to numeric)
if 'host_since' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['host_since'] = pd.to_datetime(listings['host_since'])
    listings['host_since'] = listings['host_since'].astype(int) // 10**9  # Convert to Unix timestamp

if 'first_review' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['first_review'] = pd.to_datetime(listings['first_review'])
    listings['first_review'] = listings['first_review'].astype(int) // 10**9  # Convert to Unix timestamp

if 'last_review' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['last_review'] = pd.to_datetime(listings['last_review'])
    listings['last_review'] = listings['last_review'].astype(int) // 10**9  # Convert to Unix timestamp

Importing amenities from "amenities.csv" dataset

In [135]:
amenities_dummies = pd.read_csv('amenities.csv')

In [136]:
# Concatenate the original DataFrame (excluding the old 'amenities' column) with the new boolean features
listings = pd.concat([listings.drop(columns=['amenities']), amenities_dummies], axis=1)

In [137]:
sentiment = pd.read_csv('sentiment.csv')

In [138]:
listings['id'] = listings['id'].astype(str)
sentiment['listing_id'] = sentiment['listing_id'].astype(str)

# Merge the datasets on the corresponding ID columns
merged_data = pd.merge(listings, sentiment, left_on='id', right_on='listing_id', how='left')

# Drop the duplicate 'listing_id' column if it exists
merged_data = merged_data.drop(columns=['listing_id', 'id'])

merged_data

,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,Avena,Bar,Bidet,Bikes,Blender,Boat_slip,Bowling_alley,Bread_maker,Canal_view,Candy_-_FCS100X/E_-_Horno_el\u00e9ctrico_-_2100W,Climbing_wall,Desert_view,Dishes_and_silverware,Foosball_table,Gated_property,Golf_course_view,Hammock,Hockey_rink,Ikea,Ironing_board,Kayak,Kenwood,Lake_view,Laser_tag,Lavandin,Lockbox,Marca_blanca,Marina_view,Media_room,Mini_golf,Mosquito_net,Mountain_view,Nevera_y_congelador,Noise_decibel_monitors_on_property,Palmolive,Pantene,Piano,Private_sauna,Property_manager,Record_player,Rice_maker,River_view,Samsung,Sauna,Sea_view,Sempre_hi_ha_xamp\u00fa,Shared_sauna,Siemens_Inoxidable_con_Vapor,Skate_ramp,Ski-in/Ski-out,Sonos_(Spotify_Connect,Sony,Sunstech,Table_corner_guards,Theme_room,Toaster,Valley_view,Vineyard_view,Waterfront,1_day_a_week_-_available_at_extra_cost,2_days_a_week_-_available_at_extra_cost,2-5_years_old,4_days_a_week_-_available_at_extra_cost,5_days_a_week_-_available_at_extra_cost,5-10_years_old,AirPlay,Apple,Fructis,Inox,Nivea,PS2,Puerta_desmontable_de_doble_cristal_con_efecto_espejo,Thursday_-_available_at_extra_cost,Tuesday,Wednesday,Unnamed: 75,and_10+_years_old,and_5-10_years_old,and_PS3,and_dresser,and_wardrobe,closet,every_day_-_available_at_extra_cost,every_day_-_included_with_your_stay,free_weights,olympic-sized,peque\u00f1a,rowing,wardrobe,water_slide,wood-burning,wifi_aggregated,shampoo_aggregated,body_soap_aggregated,tv_aggregated,refrigerator_aggregated,oven_aggregated,stove_aggregated,sound_system_aggregated,bbq_aggregated,games_aggregated,baby_aggregated,baking_aggregated,booster_seat_aggregated,ceiling_fan_aggregated,changing_table_aggregated,city_skyline_view_aggregated,clothing_storage_aggregated,coffee_aggregated,crib_aggregated,dryer_aggregated,charger_aggregated,dove_aggregated,elevator_aggregated,essentials_aggregated,ethernet_aggregated,exercise_equipment_aggregated,cameras_aggregated,fire_extinguisher_aggregated,first_kit_aggregated,fire_pit_aggregated,high_chair_aggregated,carport_aggregated,parking_aggregated,washer_aggregated,freezer_aggregated,garden_aggregated,housekeeping_aggregated,keypad_aggregated,kitchen_aggregated,lock_on_bedroom_aggregated,cleaning_aggregated,pool_aggregated,long_term_aggregated,luggage_dropoff_aggregated,microwave_aggregated,outdoor_aggregated,ocean_view_aggregated,gym_aggregated,hot_tub_aggregated,bay_aggregated,garage_aggregated,resort_aggregated,hangers_aggregated,heating_aggregated,host_greets_aggregated,hot_water_aggregated,pets_aggregated,private_entrance_aggregated,balcony_aggregated,safe_aggregated,self_check_in_aggregated,single_level_home_aggregated,smoke_alarm_aggregated,smoking_allowed_aggregated,sun_loungers_aggregated,prime_video_aggregated,chromecast_aggregated,dvd_aggregated,monday_to_friday_aggregated,disney_aggregated,netflix_aggregated,roku_aggregated,saturday_aggregated,sunday_aggregated,bathtub_aggregated,bed_linens_aggregated,books_and_rea

In [139]:
from sklearn.preprocessing import OneHotEncoder

# Identify numeric and categorical columns
numeric_features = merged_data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = merged_data.select_dtypes(include=['object', 'category']).columns

# Encode categorical features
encoder = OneHotEncoder(drop='first', sparse=False)
listings_encoded = pd.DataFrame(encoder.fit_transform(merged_data[categorical_features]), columns=encoder.get_feature_names_out(categorical_features), index=merged_data.index)

# Drop original categorical columns and concatenate the encoded ones
listings = merged_data.drop(columns=categorical_features)
listings = pd.concat([listings, listings_encoded], axis=1)

listings

/Users/krys/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


host_since  host_response_rate  host_acceptance_rate  \
0      1321401600               100.0                  88.0   
1      1321401600                98.0                  97.0   
2      1263859200                95.0                  85.0   
3      1321747200                 NaN                  98.0   
4      1268092800               100.0                  87.0   
...           ...                 ...                   ...   
18920  1460073600               100.0                  48.0   
18921  1589500800                99.0                  98.0   
18922  1589500800                99.0                  98.0   
18923  1568246400                99.0                  99.0   
18924  1684108800                87.0                  88.0   

       host_is_superhost  host_listings_count  host_total_listings_count  \
0                    0.0                  5.0                        7.0   
1                    0.0                 36.0                      112.0   
2                    0.0                 41.0                       45.0   
3                    1.0                  2.0                        2.0   
4                    0.0                  6.0                       13.0   
...                  ...                  ...                        ...   
18920                0.0                 53.0                       93.0   
18921                0.0                702.0                      714.0   
18922                0.0                702.0                      714.0   
18923                0.0                 24.0                      119.0   
18924                0.0                 17.0                       20.0   

       host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                       1.0                     1.0  41.401810   2.176100   
1                       1.0                     1.0  41.379360   2.173760   
2                       1.0                     1.0  41.405560   2.172620   
3                       1.0                     1.0  41.383470   2.158020   
4                       1.0                     1.0  41.412432   2.219750   
...                     ...                     ...        ...        ...   
18920                   1.0                     1.0  41.377598   2.188911   
18921                   1.0                     1.0  41.389550   2.178390   
18922                   1.0                     1.0  41.389480   2.177006   
18923                   1.0                     1.0  41.379724   2.178621   
18924                   1.0                     1.0  41.379920   2.125640   

       accommodates  bathrooms  bedrooms  beds  price  minimum_nights  \
0                 2        1.0       1.0   1.0  176.0               7   
1                 6        1.5       4.0   5.0   74.0              32   
2                 8        2.0       3.0   6.0  257.0               1   
3                 8        NaN       3.0   NaN    NaN               4   
4                 5        2.0       3.0   4.0  249.0               3   
...             ...        ...       ...   ...    ...             ...   
18920             2        1.0       1.0   1.0   86.0              32   
18921             4        2.0       2.0   2.0  167.0              31   
18922             4        2.0       2.0   2.0  165.0              31   
18923             2        1.0       1.0   1.0   44.0               5   
18924             2        1.0       1.0   1.0   95.0               2   

       maximum_nights  minimum_minimum_nights  maximum_minimum_nights  \
0                 760                       7                       7   
1                 760                      32                      32   
2                1125                       1                       4   
3                1125                       3                       3   
4                  32                       1                       7   
...               ...                     ...                     ...   
18920             345   

Missing values handling through Regression Imputation

In [140]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Function to fill missing values using linear regression
def fill_missing_values(df):
    df_filled = df.copy()
    
    for column in df.columns:
        if df[column].isnull().sum() > 0:

            # Separate the data into columns with and without missing values
            df_not_null = df_filled[df_filled[column].notnull()]
            df_null = df_filled[df_filled[column].isnull()]
            
            if df_not_null.empty or df_null.empty:
                # Skip column if there are no missing values
                continue
            
            # Define the model
            model = LinearRegression()
            
            # Ensure that X_train and X_test have no missing values
            X_train = df_not_null.drop(columns=[column])
            y_train = df_not_null[column]
            
            # Check for any NaNs in the training data
            if X_train.isnull().any().any():
                # If training data for the column contains NaNs, imput missing values before training
                imputer = SimpleImputer(strategy='mean')
                X_train = imputer.fit_transform(X_train)
            
            X_test = df_null.drop(columns=[column])
            
            if X_test.isnull().any().any():
                # If test data for the column contains NaNs, imput missing values before prediction
                X_test = imputer.transform(X_test)
            
            # Train the model
            model.fit(X_train, y_train)
            
            # Predict the missing values
            predicted_values = model.predict(X_test)
            
            # Assign the predicted values to the original dataframe
            df_filled.loc[df_filled[column].isnull(), column] = predicted_values
    
    return df_filled


# Apply the function to fill missing values in the numeric data
listings_filled = fill_missing_values(listings)

In [141]:
listings_filled

host_since  host_response_rate  host_acceptance_rate  \
0      1321401600          100.000000                  88.0   
1      1321401600           98.000000                  97.0   
2      1263859200           95.000000                  85.0   
3      1321747200           24.668163                  98.0   
4      1268092800          100.000000                  87.0   
...           ...                 ...                   ...   
18920  1460073600          100.000000                  48.0   
18921  1589500800           99.000000                  98.0   
18922  1589500800           99.000000                  98.0   
18923  1568246400           99.000000                  99.0   
18924  1684108800           87.000000                  88.0   

       host_is_superhost  host_listings_count  host_total_listings_count  \
0                    0.0                  5.0                        7.0   
1                    0.0                 36.0                      112.0   
2                    0.0                 41.0                       45.0   
3                    1.0                  2.0                        2.0   
4                    0.0                  6.0                       13.0   
...                  ...                  ...                        ...   
18920                0.0                 53.0                       93.0   
18921                0.0                702.0                      714.0   
18922                0.0                702.0                      714.0   
18923                0.0                 24.0                      119.0   
18924                0.0                 17.0                       20.0   

       host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                       1.0                     1.0  41.401810   2.176100   
1                       1.0                     1.0  41.379360   2.173760   
2                       1.0                     1.0  41.405560   2.172620   
3                       1.0                     1.0  41.383470   2.158020   
4                       1.0                     1.0  41.412432   2.219750   
...                     ...                     ...        ...        ...   
18920                   1.0                     1.0  41.377598   2.188911   
18921                   1.0                     1.0  41.389550   2.178390   
18922                   1.0                     1.0  41.389480   2.177006   
18923                   1.0                     1.0  41.379724   2.178621   
18924                   1.0                     1.0  41.379920   2.125640   

       accommodates  bathrooms  bedrooms      beds       price  \
0                 2   1.000000       1.0  1.000000  176.000000   
1                 6   1.500000       4.0  5.000000   74.000000   
2                 8   2.000000       3.0  6.000000  257.000000   
3                 8   2.012331       3.0  5.637927  414.019055   
4                 5   2.000000       3.0  4.000000  249.000000   
...             ...        ...       ...       ...         ...   
18920             2   1.000000       1.0  1.000000   86.000000   
18921             4   2.000000       2.0  2.000000  167.000000   
18922             4   2.000000       2.0  2.000000  165.000000   
18923             2   1.000000       1.0  1.000000   44.000000   
18924             2   1.000000       1.0  1.000000   95.000000   

       minimum_nights  maximum_nights  minimum_minimum_nights  \
0                   7             760                       7   
1                  32             760                      32   
2                   1            1125                       1   
3                   4            1125                       3   
4                   3              32                       1   
...               ...             ...                     ...   
18920              32             345                      32   
18921              31             330                      31   
18922              31             

In [142]:
from sklearn.preprocessing import StandardScaler

# Scale the numeric data
scaler = StandardScaler()
listings_scaled = scaler.fit_transform(listings_filled)

# Convert the scaled data back to a DataFrame
listings = pd.DataFrame(listings_scaled, columns=listings_filled.columns)

listings

host_since  host_response_rate  host_acceptance_rate  \
0       -1.048200            0.578447              0.248614   
1       -1.048200            0.510899              0.623327   
2       -1.402525            0.409578              0.123710   
3       -1.046072           -1.965791              0.664962   
4       -1.376456            0.578447              0.206979   
...           ...                 ...                   ...   
18920   -0.194311            0.578447             -1.416776   
18921    0.602652            0.544673              0.664962   
18922    0.602652            0.544673              0.664962   
18923    0.471776            0.544673              0.706597   
18924    1.185213            0.139388              0.248614   

       host_is_superhost  host_listings_count  host_total_listings_count  \
0              -0.500223            -0.392722                  -0.420800   
1              -0.500223            -0.143146                   0.309237   
2              -0.500223            -0.102892                  -0.156596   
3               2.007882            -0.416874                  -0.455564   
4              -0.500223            -0.384671                  -0.379084   
...                  ...                  ...                        ...   
18920          -0.500223            -0.006283                   0.177135   
18921          -0.500223             5.218692                   4.494782   
18922          -0.500223             5.218692                   4.494782   
18923          -0.500223            -0.239756                   0.357906   
18924          -0.500223            -0.296112                  -0.330414   

       host_has_profile_pic  host_identity_verified  latitude  longitude  \
0                  0.151349                0.302518  0.678299   0.514940   
1                  0.151349                0.302518 -0.898304   0.382201   
2                  0.151349                0.302518  0.941651   0.317533   
3                  0.151349                0.302518 -0.609670  -0.510667   
4                  0.151349                0.302518  1.424234   2.991052   
...                     ...                     ...       ...        ...   
18920              0.151349                0.302518 -1.022016   1.241674   
18921              0.151349                0.302518 -0.182688   0.644843   
18922              0.151349                0.302518 -0.187627   0.566331   
18923              0.151349                0.302518 -0.872725   0.657959   
18924              0.151349                0.302518 -0.858977  -2.347457   

       accommodates  bathrooms  bedrooms      beds     price  minimum_nights  \
0         -0.617899  -0.475698 -0.650489 -0.574876 -0.013791       -0.262852   
1          1.233035   0.155240  1.905760  1.358539 -0.347600        0.502768   
2          2.158502   0.786178  1.053677  1.841892  0.251293       -0.446601   
3          2.158502   0.801739  1.053677  1.666883  0.765159       -0.354727   
4          0.770302   0.786178  1.053677  0.875185  0.225112       -0.385352   
...             ...        ...       ...       ...       ...             ...   
18920     -0.617899  -0.475698 -0.650489 -0.574876 -0.308328        0.502768   
18921      0.307568   0.786178  0.201594 -0.091522 -0.043245        0.472143   
18922      0.307568   0.786178  0.201594 -0.091522 -0.049790        0.472143   
18923     -0.617899  -0.475698 -0.650489 -0.574876 -0.445779       -0.324102   
18924     -0.617899  -0.475698 -0.650489 -0.574876 -0.278874       -0.415976   

       maximum_nights  minimum_minimum_nights  maximum_minimum_nights  \
0            0.513224               -0.247892               -0.289146   
1            0.513224                0.573885                0.258814   
2            1.373950               -0.445119               -0.354901   
3            1.373950               -0.379377               -0.376819   
4           -1.203511               -0.445119               -0.289146   
...               ...    

<h3>Feature selection with Lasso Regularization</h3>

In [143]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Separate features and target
X = listings.drop(columns=["price"])
y = listings["price"]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and fit Lasso model
lasso = Lasso(alpha=0.01)  # alpha is the regularization strength
lasso.fit(X_train_scaled, y_train)

# Make predictions
y_pred = lasso.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# View the coefficients
print("Lasso Coefficients:", lasso.coef_)

Mean Squared Error: 0.3213206972226468
Lasso Coefficients: [ 0.         -0.00896448 -0.         ...  0.         -0.
  0.        ]


In [144]:
import numpy as np

# Get the feature names if you have them
feature_names = np.array(X.columns)  # if X is a DataFrame

# Get the coefficients from the Lasso model
coefficients = lasso.coef_

# Identify the features with non-zero coefficients
selected_features = feature_names[coefficients != 0]

print("Selected Features:", selected_features)

Selected Features: ['host_response_rate' 'latitude' 'accommodates' 'bathrooms' 'bedrooms'
 'minimum_nights' 'minimum_minimum_nights' 'maximum_minimum_nights'
 'minimum_nights_avg_ntm' 'availability_30' 'availability_60'
 'availability_90' 'number_of_reviews_ltm' 'number_of_reviews_l30d'
 'last_review' 'instant_bookable'
 'calculated_host_listings_count_entire_homes' 'Bidet' 'Mountain_view'
 'Piano' 'Private_sauna' 'Sauna' 'Waterfront' 'wifi_aggregated'
 'shampoo_aggregated' 'tv_aggregated' 'games_aggregated' 'baby_aggregated'
 'baking_aggregated' 'changing_table_aggregated'
 'clothing_storage_aggregated' 'crib_aggregated' 'elevator_aggregated'
 'ethernet_aggregated' 'high_chair_aggregated' 'freezer_aggregated'
 'kitchen_aggregated' 'lock_on_bedroom_aggregated' 'pool_aggregated'
 'ocean_view_aggregated' 'gym_aggregated' 'heating_aggregated'
 'private_entrance_aggregated' 'safe_aggregated'
 'single_level_home_aggregated' 'smoke_alarm_aggregated'
 'sun_loungers_aggregated' 'books_and_read

In [145]:
X_selected = X[selected_features]

print("Number of features in updated dataset:", X_selected.shape[1])


Number of features in updated dataset: 107


In [146]:
X_selected.to_csv('X.csv', index=False)

y.to_csv('y.csv', index=False)